In [1]:
from cgi import test
import os
from collections import Counter
import numpy as np
import json
import torch
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image



/tmp/ipykernel_37686/4114127383.py:1: DeprecationWarning: 'cgi' is deprecated and slated for removal in Python 3.13
  from cgi import test
/home/dnull/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

image_path = "./data/deepfashion-multimodal/images/"
train_text = "./data/deepfashion-multimodal/train_captions.json"
test_text = "./data/deepfashion-multimodal/test_captions.json"


class MyDataset(Dataset):
    def __init__(self, image_paths, train_captions, test_captions, transform=None):
        self.image_paths = image_paths
        self.train_captions = train_captions
        self.test_captions = test_captions
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        idx_name = self.image_paths[idx]
        file_name = idx_name.split("/")[-1]
        image = Image.open(idx_name).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, self.train_captions[file_name]
    
    def get_train_captions(self):
        return self.train_captions

In [3]:
# read data
train_captions = json.load(open(train_text, 'r'))
test_captions = json.load(open(test_text, 'r'))
image_paths = []
# add from train_captions
for key in train_captions.keys():
    image_paths.append(image_path + key)

In [4]:
transform = transforms.Compose(
    [
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

In [5]:
image_dic = train_captions.keys()
image_descriptions = train_captions.values()

# build vocabulary
vocab = Counter()
for description in image_descriptions:
    vocab.update(description.split())

# remove words that occur less than threshold
threshold = -1
words = [word for word, count in vocab.items() if count >= threshold]

# create a mapping from word to index and index to word
idx_to_word = {idx: word for idx, word in enumerate(words, 1)}

# add the start and end token to the vocabulary
idx_to_word[0] = "<pad>"
idx_to_word[len(idx_to_word)] = "<end>"
idx_to_word[len(idx_to_word)] = "<start>"

# add the end to the end pos
for key, description in train_captions.items():
    train_captions[key] = "<start> " + description + " <end>"

# pad the descriptions with <pad>
max_length = max(len(description.split()) for description in image_descriptions)
for key, description in train_captions.items():
    train_captions[key] = description + " <pad>" * (
        max_length - len(description.split())
    )
for key, description in test_captions.items():
    test_captions[key] = description + " <pad>" * (
        max_length - len(description.split())
    )

word_to_idx = {word: idx for idx, word in idx_to_word.items()}
vocab_size_len = len(idx_to_word)


# convert each word to its index
train_captions = {
    key: [word_to_idx[word] for word in value.split()]
    for key, value in train_captions.items()
}
test_captions = {
    key: [word_to_idx[word] for word in value.split()]
    for key, value in test_captions.items()
}


for key, value, idx in zip(train_captions.keys(), train_captions.values(), range(5)):
    print(key, value, idx)
    if idx == 4:
        break


WOMEN-Jackets_Coats-id_00005611-01_4_full.jpg [159, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 13, 14, 15, 16, 17, 1, 18, 3, 16, 14, 5, 19, 1, 8, 16, 20, 9, 15, 4, 10, 11, 12, 21, 22, 23, 24, 25, 26, 27, 28, 7, 8, 9, 29, 12, 21, 30, 16, 31, 32, 33, 34, 35, 36, 21, 30, 4, 37, 158, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 0
WOMEN-Tees_Tanks-id_00005033-03_4_full.jpg [159, 38, 39, 40, 4, 41, 6, 42, 8, 9, 43, 12, 44, 4, 32, 45, 46, 1, 47, 24, 32, 5, 48, 1, 49, 50, 28, 20, 8, 9, 10, 11, 12, 1, 22, 24, 32, 51, 158, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 1
WOMEN-Rompers_Jumpsuits-id_00000245-01_1_front.jpg [159, 38, 39, 52, 4, 41, 6, 7, 8, 9, 10, 11, 12, 44, 4, 32, 53, 46, 21, 54, 24, 32, 5, 55, 1, 56, 50, 28, 7, 8, 9, 10, 11, 12, 57, 16, 32, 33, 34, 35, 36, 1, 22, 24, 32, 58, 57, 16, 25, 59, 34, 35, 60, 158, 0, 

In [6]:

for key, value in train_captions.items():
    train_captions[key] = torch.tensor(value, dtype=torch.long)

# make dataset
dataset = MyDataset(image_paths, train_captions, test_captions, transform)

for i in range(1):
    print(dataset[i][0].shape, dataset[i][1].shape)
    print(dataset[i][0], dataset[i][1])

torch.Size([3, 256, 256]) torch.Size([95])
tensor([[[2.2318, 2.2318, 2.2318,  ..., 2.1804, 2.1804, 2.1804],
         [2.2318, 2.2147, 2.2318,  ..., 2.1804, 2.1804, 2.1804],
         [2.2318, 2.2318, 2.2318,  ..., 2.1804, 2.1804, 2.1804],
         ...,
         [2.1804, 2.1633, 2.1633,  ..., 2.0777, 2.0777, 2.0777],
         [2.1804, 2.1633, 2.1633,  ..., 2.0948, 2.0777, 2.0777],
         [2.1804, 2.1804, 2.1633,  ..., 2.0948, 2.0948, 2.0777]],

        [[2.4111, 2.4111, 2.4111,  ..., 2.3585, 2.3585, 2.3585],
         [2.4111, 2.3936, 2.4111,  ..., 2.3585, 2.3585, 2.3585],
         [2.4111, 2.4111, 2.4111,  ..., 2.3585, 2.3585, 2.3585],
         ...,
         [2.3235, 2.3410, 2.3410,  ..., 2.2185, 2.2185, 2.2185],
         [2.3235, 2.3410, 2.3410,  ..., 2.2360, 2.2185, 2.2185],
         [2.3235, 2.3410, 2.3410,  ..., 2.2360, 2.2360, 2.2185]],

        [[2.6226, 2.6226, 2.6226,  ..., 2.5703, 2.5703, 2.5703],
         [2.6226, 2.6051, 2.6226,  ..., 2.5703, 2.5703, 2.5703],
         [2.622

In [7]:
import torch
import torch.nn as nn
import torchvision.models as models
from torch.nn import Transformer


# 1. 使用预训练的 CNN 提取特征
class FeatureExtractorCNN(nn.Module):
    def __init__(self):
        super(FeatureExtractorCNN, self).__init__()
        
        self.resnet = models.resnet18(pretrained=True)
        self.resnet = nn.Sequential(*list(self.resnet.children())[:-1])

    def forward(self, images):
        # resize to 3 times larger
        batch_size = images.shape[0]
        images = nn.functional.interpolate(
            images, scale_factor=3, mode="bilinear", align_corners=False
        )
        images = images.unfold(2, 256, 256).unfold(3, 256, 256)
        images = images.contiguous().view(-1, 3, 256, 256)

        # 提取特征
        features = self.resnet(images)
        features = features.view(batch_size, 9, -1)
        return features


# 2. 构建 Transformer 模型
class ImageCaptioningTransformer(nn.Module):
    def __init__(self, emb_dim, nhead, nhid, nlayers, vocab_size, max_seq_length):
        super(ImageCaptioningTransformer, self).__init__()
        self.pos_encoder = PositionalEncoding(emb_dim)
        self.transformer = Transformer(
            d_model=emb_dim,
            nhead=nhead,
            dim_feedforward=nhid,
            num_encoder_layers=nlayers,
            num_decoder_layers=nlayers,
            batch_first=False,
            dropout=0.2,
        )
        self.decoder = nn.Linear(emb_dim, vocab_size)
        self.max_seq_length = max_seq_length
        self.embedding = nn.Embedding(vocab_size, emb_dim)

    def forward(self, src, tgt, src_mask=None, tgt_mask=None, src_padding_mask=None,
                tgt_padding_mask=None, memory_mask=None):
        src = self.pos_encoder(src)  # 添加位置编码
        tgt = self.embedding(tgt)  # 词嵌入
        tgt = self.pos_encoder(tgt)  # 添加位置编码

        output = self.transformer(src, tgt, src_mask, tgt_mask, None, src_padding_mask,
                tgt_padding_mask, memory_mask)
        output = self.decoder(output)
        return output


# 辅助类：位置编码
import torch
import torch.nn as nn
import math


class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=1000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # 创建位置编码
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model)
        )
        pe = torch.zeros(max_len, d_model)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer("pe", pe)

    def forward(self, x):
        x = x + self.pe[: x.size(0), :]
        return self.dropout(x)





In [8]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device='cuda')) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device='cuda').type(torch.bool)

    src_padding_mask = (src == 0).transpose(0, 1)
    tgt_padding_mask = (tgt == 0).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [9]:
dataloader = DataLoader(dataset, batch_size=16, shuffle=True, num_workers=4)
# check dataset

for i, (image, caption) in enumerate(dataloader):
    if i == 0:
        break

# torch.Size([2, 3, 256, 256]) torch.Size([2, 93])


# 定义模型参数
emb_dim = 512  # 嵌入维度
nhead = 8  # 多头注意力的头数
nhid = 1024  # 前馈网络的维度
nlayers = 3  # 编码器和解码器层的数量
vocab_size = vocab_size_len  # 词汇表大小
max_seq_length = 512  # 最大序列长度

cnn = FeatureExtractorCNN()
transformer = ImageCaptioningTransformer(
    emb_dim, nhead, nhid, nlayers, vocab_size, max_seq_length
)

# check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
cnn = cnn.to(device)
transformer = transformer.to(device)

# train
epoch = 5
lr = 0.001
optimizer = torch.optim.Adam(transformer.parameters(), lr=lr)

vocab_size = 160
pad_index = 0

# 使用权重创建损失函数
criterion = nn.CrossEntropyLoss(ignore_index=0)

for e in range(epoch):
    for i, (image, caption) in enumerate(dataloader):
        image = image.to(device)
        caption = caption.to(device)
        
        optimizer.zero_grad()
        features = cnn(image)
        
        features = features.transpose(0, 1)
        caption = caption.transpose(0, 1)
        
        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(features, caption[:-1, :])
        

        output = transformer(features, caption[:-1, :], src_mask, tgt_mask, None, tgt_padding_mask)
        
        loss = criterion(output.reshape(-1, 160), caption[1:, :].reshape(-1))
        
        #check each pos
        temp = 0
        pad_num = 0
        pad_num_2 = 0

        loss.backward()
        optimizer.step()
        if i % 10 == 0:
            print("epoch: {}, step: {}, loss: {}".format(e, i, loss.item()))
            
            #print output
            # print("output: ", end="")
            # for i in range(92):
            #     print(idx_to_word[torch.argmax(output[i])], end=" ")
            # print('\n')
    torch.save(transformer.state_dict(), "./model/transformer_{}.pth".format(e))

/home/dnull/miniconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/dnull/miniconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/dnull/miniconda3/lib/python3.11/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is 

cuda


/home/dnull/miniconda3/lib/python3.11/site-packages/torch/nn/functional.py:5076: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


epoch: 0, step: 0, loss: 5.289523601531982
epoch: 0, step: 10, loss: 4.152801036834717
epoch: 0, step: 20, loss: 4.036811828613281
epoch: 0, step: 30, loss: 2.8322768211364746
epoch: 0, step: 40, loss: 1.4876823425292969
epoch: 0, step: 50, loss: 1.0513161420822144
epoch: 0, step: 60, loss: 0.9210466742515564
epoch: 0, step: 70, loss: 0.906835675239563
epoch: 0, step: 80, loss: 0.7109276652336121
epoch: 0, step: 90, loss: 0.8254294991493225
epoch: 0, step: 100, loss: 0.685184895992279
epoch: 0, step: 110, loss: 0.68462073802948
epoch: 0, step: 120, loss: 0.6383317112922668
epoch: 0, step: 130, loss: 0.6734803915023804
epoch: 0, step: 140, loss: 0.6619338393211365
epoch: 0, step: 150, loss: 0.6647012829780579
epoch: 0, step: 160, loss: 0.6635385155677795
epoch: 0, step: 170, loss: 0.6401270627975464
epoch: 0, step: 180, loss: 0.6715878844261169
epoch: 0, step: 190, loss: 0.6044638752937317
epoch: 0, step: 200, loss: 0.6118202209472656
epoch: 0, step: 210, loss: 0.6002620458602905
epoch:

In [17]:
def generate_caption(model, image_features, word2idx, idx2word,  max_length=93):

    outputs = [word2idx["<start>"]]

    for i in range(max_length - 1):
        
        tgt_mask = (torch.triu(torch.ones(i + 1, i + 1)) == 1).transpose(0, 1)
        tgt_mask = tgt_mask.float().masked_fill(tgt_mask == 0, float('-inf')).masked_fill(tgt_mask == 1, float(0.0))
        
        with torch.no_grad():
            
            out = model(image_features, torch.tensor(outputs).unsqueeze(1).to(device), tgt_mask=tgt_mask.to(device))
        
        next_word = out.argmax(dim=2)[-1].item()

        # 检查是否达到结束标记
        if next_word == word2idx['<end>']:
            break  # 一旦生成<end>标记，立即停止生成

        outputs.append(next_word)
        
        # print output
        print("output: ", end="")
        for i in range(len(outputs)):
            print(idx2word[outputs[i]], end=" ")
        print('\n')

    # 转换序列为文字
    caption = [idx2word[idx] for idx in outputs]

    return ' '.join(caption)


In [18]:
# load model
transformer.load_state_dict(torch.load("./model/transformer_0.pth"))
transformer.eval()

# test
test_data = dataset[7]
print(test_data[0].shape, test_data[1].shape)

test_image = test_data[0].unsqueeze(0).to(device)
test_feature = cnn(test_image)

test_feature = test_feature.transpose(0, 1)

output = generate_caption(transformer, test_feature, word_to_idx, idx_to_word)

# output = transformer(test_feature, test_feature, None, None, None, None)

print(output)

# idx2word
idx2word = {idx: word for idx, word in enumerate(words, 1)}
idx2word[0] = "<pad>"
idx2word[len(idx2word)] = "<end>"


torch.Size([3, 256, 256]) torch.Size([95])
output: <start> The 

output: <start> The person 

output: <start> The person is 

output: <start> The person is wearing 

output: <start> The person is wearing a 

output: <start> The person is wearing a sleeveless 

output: <start> The person is wearing a sleeveless tank 

output: <start> The person is wearing a sleeveless tank shirt 

output: <start> The person is wearing a sleeveless tank shirt with 

output: <start> The person is wearing a sleeveless tank shirt with solid 

output: <start> The person is wearing a sleeveless tank shirt with solid color 

output: <start> The person is wearing a sleeveless tank shirt with solid color patterns. 

output: <start> The person is wearing a sleeveless tank shirt with solid color patterns. The 

output: <start> The person is wearing a sleeveless tank shirt with solid color patterns. The tank 

output: <start> The person is wearing a sleeveless tank shirt with solid color patterns. The tank shirt 

